# Executing Quantum Photonic Circuits 

In CUDA-Q, there are 2 ways in which one can execute quantum photonic kernels: 

1. `sample`: yields measurement counts 
3. `get_state`: yields the quantum statevector of the computation 

## Sample

Quantum states collapse upon measurement and hence need to be sampled many times to gather statistics. The CUDA-Q `sample` call enables this: 



In [ ]:
import cudaq
import numpy as np

qumode_count = 2

# Define the simulation target.
cudaq.set_target("orca-photonics")

# Define a quantum kernel function.


@cudaq.kernel
def kernel(qumode_count: int):
    level = qumode_count + 1
    qumodes = [qudit(level) for _ in range(qumode_count)]

    # Apply the create gate to the qumodes.
    for i in range(qumode_count):
        create(qumodes[i])  # |00⟩ -> |11⟩

    # Apply the beam_splitter gate to the qumodes.
    beam_splitter(qumodes[0], qumodes[1], np.pi / 6)

    # measure all qumodes
    mz(qumodes)


result = cudaq.sample(kernel, qumode_count, shots_count=1000)

print(result)


## Get state

The `get_state` function gives us access to the quantum statevector of the computation.

In [ ]:
import cudaq
import numpy as np

qumode_count = 2

# Define the simulation target.
cudaq.set_target("orca-photonics")

# Define a quantum kernel function.


@cudaq.kernel
def kernel(qumode_count: int):
    level = qumode_count + 1
    qumodes = [qudit(level) for _ in range(qumode_count)]

    # Apply the create gate to the qumodes.
    for i in range(qumode_count):
        create(qumodes[i])  # |00⟩ -> |11⟩

    # Apply the beam_splitter gate to the qumodes.
    beam_splitter(qumodes[0], qumodes[1], np.pi / 6)

    # measure some of all qumodes if need to be measured
    # mz(qumodes)


# Compute the statevector of the kernel
result = cudaq.get_state(kernel, qumode_count)

print(np.array(result))

The statevector generated by the `get_state` command follows little-endian convention for associating numbers with their digit string representations, which places the least significant digit on the right. That is, for the example of a 2-qumode system of level 3 (in which possible states are 0, 1, and 2), we have the following translation between integers and digit string:
$$\begin{matrix} 
\text{Integer} & \text{digit string representation}\\
& \text{least significant bit on right}\\
0 = \textcolor{blue}{0}*3^1 + \textcolor{red}{0}*3^0 & \textcolor{blue}{0}\textcolor{red}{0} \\
1 = \textcolor{blue}{0}*3^1 + \textcolor{red}{1}*3^0 & \textcolor{blue}{0}\textcolor{red}{1}\\
2 = \textcolor{blue}{0}*3^1 + \textcolor{red}{2}*3^0 & \textcolor{blue}{0}\textcolor{red}{2}\\
3 = \textcolor{blue}{1}*3^1 + \textcolor{red}{0}*3^0 & \textcolor{blue}{1}\textcolor{red}{0} \\
4 = \textcolor{blue}{1}*3^1 + \textcolor{red}{1}*3^0 & \textcolor{blue}{1}\textcolor{red}{1} \\
5 = \textcolor{blue}{1}*3^1 + \textcolor{red}{2}*3^0 & \textcolor{blue}{1}\textcolor{red}{2} \\
6 = \textcolor{blue}{2}*3^1 + \textcolor{red}{0}*3^0 & \textcolor{blue}{2}\textcolor{red}{0} \\
7 = \textcolor{blue}{2}*3^1 + \textcolor{red}{1}*3^0 & \textcolor{blue}{2}\textcolor{red}{1} \\
8 = \textcolor{blue}{2}*3^1 + \textcolor{red}{2}*3^0 & \textcolor{blue}{2}\textcolor{red}{2} 
\end{matrix}
$$



## Parallelization Techniques

The most intensive task in the computation is the execution of the quantum photonic kernel hence each execution function: `sample`, and `get_state` can be parallelized given access to multiple quantum processing units (multi-QPU). We emulate each QPU with a CPU.

In [ ]:
print(cudaq.__version__)